# Second model 

This model will try to predict wether pe is present or not on a full image (RGB level)

## Imports

In [1]:
import numpy as np
from balance import *
from torchsummary import summary

In [2]:
import torch
torch.cuda.empty_cache()

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

PyTorch version: 1.7.1
GPU found :)


# Load datasets

In [3]:
full_image_dataset = torch.load('datasets/full_image_dataset.pt')

## Simple CNN from scratch

In [4]:
class ModelV2(nn.Module):
    
    def __init__(self, input_channels):
        """Simple Convnet with 2 convolution layer + pooling, with 2 fully connected at the end"""
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.fc1 = nn.Linear(32*62*62 , 120)
        self.fc2 = nn.Linear(120, 2)
        
        
    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32*62*62) # Flatten
        x = F.relu(self.fc1(x))
        
        # Binary classification
        out = torch.sigmoid(self.fc2(x))
        
        return out

In [ ]:
summary(ModelV2(3), (3, 256, 256))

## Split data into train and test

In [1]:
dataset = full_image_dataset
batch_size = 64
ratio=0.2

train_loader, test_loader = train_test_split(dataset, ratio, batch_size)

NameError: name 'channel0_dataset' is not defined

## Model Training

In [6]:
num_epoch = 5
step_count = len(train_loader)
loss_function = nn.CrossEntropyLoss()
model = ModelV2(input_channels=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(num_epoch):
    for i, sample in enumerate(train_loader):
        
        image = sample['image'].to(device, dtype=torch.float)
        target = sample['target'].to(device)
        
        # Reset gradiant
        optimizer.zero_grad()
        
        # Forward pass
        pred = model(image)
        
        # Compute loss
        loss = loss_function(pred, target)
        
        # Backprop
        loss.backward()
        optimizer.step()
        
        # Debug
        if((i+1) % 100 == 0):
            print(
                        f"Epoch [{epoch + 1}/{num_epoch}]"
                        f", step [{i + 1}/{step_count}]"
                        f", loss: {loss.item():.4f}"
                    )

Epoch [1/5], step [100/1254], loss: 0.7102
Epoch [1/5], step [200/1254], loss: 0.6763
Epoch [1/5], step [300/1254], loss: 0.7020
Epoch [1/5], step [400/1254], loss: 0.7025
Epoch [1/5], step [500/1254], loss: 0.6835
Epoch [1/5], step [600/1254], loss: 0.6842
Epoch [1/5], step [700/1254], loss: 0.6846
Epoch [1/5], step [800/1254], loss: 0.7022
Epoch [1/5], step [900/1254], loss: 0.6849
Epoch [1/5], step [1000/1254], loss: 0.6852
Epoch [1/5], step [1100/1254], loss: 0.7015
Epoch [1/5], step [1200/1254], loss: 0.7010
Epoch [2/5], step [100/1254], loss: 0.6844
Epoch [2/5], step [200/1254], loss: 0.6852
Epoch [2/5], step [300/1254], loss: 0.6842
Epoch [2/5], step [400/1254], loss: 0.7021
Epoch [2/5], step [500/1254], loss: 0.6833
Epoch [2/5], step [600/1254], loss: 0.6839
Epoch [2/5], step [700/1254], loss: 0.6847
Epoch [2/5], step [800/1254], loss: 0.6847
Epoch [2/5], step [900/1254], loss: 0.7016
Epoch [2/5], step [1000/1254], loss: 0.6863
Epoch [2/5], step [1100/1254], loss: 0.6866
Epoch 

In [7]:
# Save model 
torch.save(model.state_dict(), './models/model_v2.h5')

## Test the model

In [8]:
# Load model for evaluation
trained_model = ModelV2(input_channels=3)
trained_model.load_state_dict(torch.load('./models/model_v2.h5'))
trained_model.to(device)

ModelV1(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=123008, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=2, bias=True)
)

In [10]:
correct = 0
total = 0
with torch.no_grad():
    for sample in test_loader:
        
        image = sample['image'].to(device, dtype=torch.float)
        target = sample['target'].to(device)
        
        output = model(image)
        prob, index = torch.max(output, 1)
        correct += (index == target).sum().item()

total = len(test_loader)
print(f'Accuracy of the network on the {total} test images: {100 * correct / total}')

Accuracy of the network on the 1254 test images: 50.0
